# Interactive Demo for Metrics

* command line executables: see README.md
* algorithm documentation: [metrics.py API & Algorithm Documentation](metrics.py_API_Documentation.ipynb)

...some modules and settings for this demo:

In [1]:
import os
import pprint
import pandas as pd
import numpy as np
import copy
import glob
from evo.tools import log
from evo.core import metrics
from evo.tools import file_interface
from evo.core import sync
log.configure_logging()

In [2]:
from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.tools.settings import SETTINGS

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

In [3]:
from evo.core.trajectory import PoseTrajectory3D
from evo.tools import file_interface
import numpy as np

DESC = "Combine KITTI poses and timestamps files to a TUM trajectory file"


def kitti_poses_and_timestamps_to_trajectory(poses_file, timestamp_file):
    pose_path = file_interface.read_kitti_poses_file(poses_file)
    raw_timestamps_mat = file_interface.csv_read_matrix(timestamp_file)
    error_msg = ("timestamp file must have one column of timestamps and same number of rows as the KITTI poses file")
    if len(raw_timestamps_mat) > 0 and len(raw_timestamps_mat[0]) != 1 or len(raw_timestamps_mat) != pose_path.num_poses:
        raise file_interface.FileInterfaceException(error_msg)
    try:
        timestamps_mat = np.array(raw_timestamps_mat).astype(float)
    except ValueError:
        raise file_interface.FileInterfaceException(error_msg)
    return PoseTrajectory3D(poses_se3=pose_path.poses_se3, timestamps=timestamps_mat)

In [5]:
# interactive widgets configuration
import ipywidgets

check_opts_ape = {"align_origin": True,"align": False, "correct_scale": True, "show_plot": True}
check_boxes_ape=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_ape.items()]
check_opts_rpe = {"align_origin": True,"align": False, "correct_scale": True, "all_pairs": True, "show_plot": True}
check_boxes_rpe=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_rpe.items()]
delta_input = ipywidgets.FloatText(value=1.0, description='delta', disabled=False, color='black')
delta_unit_selector=ipywidgets.Dropdown(
    options={u.value: u for u in Unit if u is not Unit.seconds},
    value=Unit.frames, description='delta_unit'
)
plotmode_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PlotMode},
    value=PlotMode.xy, description='plot_mode'
)
pose_relation_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PoseRelation},
    value=PoseRelation.translation_part, description='pose_relation'
)

ModuleNotFoundError: No module named 'ipywidgets'

In [6]:
def metrics_dataframe(reference,estimate,algorithm,dataset,trajectory,correct_scale=True,correct_only_scale=False,n=-1,use_aligned_trajectories = True,t_offset = .1):
    reference, estimate = sync.associate_trajectories(reference, estimate)
    est = copy.deepcopy(estimate)
    if use_aligned_trajectories:
        est.align(reference, correct_scale=correct_scale, correct_only_scale=correct_only_scale, n=n)

    if use_aligned_trajectories:
        data = (reference, est) 
    else:
        data = (reference, estimate)
    
    result_dict = {}
    #APE_t
    pose_relation = metrics.PoseRelation.translation_part
    ape_metric = metrics.APE(pose_relation)
    ape_metric.process_data(data)
    ape_stats = ape_metric.get_all_statistics()
    result_dict.update({(algorithm, dataset,trajectory,'ape_t'):ape_stats})
    #APE_rad
    pose_relation = metrics.PoseRelation.rotation_angle_rad
    ape_metric = metrics.APE(pose_relation)
    ape_metric.process_data(data)
    ape_stats = ape_metric.get_all_statistics()
    result_dict.update({(algorithm, dataset,trajectory,'ape_rad'):ape_stats})
    
    #RPE_t
    pose_relation = metrics.PoseRelation.translation_part
    rpe_metric = metrics.RPE(pose_relation)
    rpe_metric.process_data(data)
    rpe_stats = rpe_metric.get_all_statistics()
    result_dict.update({(algorithm, dataset,trajectory,'rpe_t'):rpe_stats})
    #RPE_rad
    pose_relation = metrics.PoseRelation.rotation_angle_rad
    rpe_metric = metrics.RPE(pose_relation)
    rpe_metric.process_data(data)
    rpe_stats = rpe_metric.get_all_statistics()
    result_dict.update({(algorithm, dataset,trajectory,'rpe_rad'):rpe_stats})
    
    result_df = pd.DataFrame(result_dict)
    return result_df

def dso_estimate_workaround(reference, estimate):
    startime = reference.timestamps[0]
    endtime  = reference.timestamps[-1]
    estimate_aux = pd.read_csv(estimate, header = None, delim_whitespace = True, names = ['t', 'x', 'y', 'z', 'qx', 'qy', 'qz', 'qw'] )
    len_estimate = len(estimate_aux['t'].tolist())
    estimate_new_ts = np.linspace(startime*1e9,endtime*1e9, num = len_estimate)
    estimate_aux['#timestamp'] = estimate_new_ts.flatten().tolist()
    estimate_aux.to_csv('temp.txt', header = True, index = False, columns = ['#timestamp', 'x', 'y', 'z', 'qw', 'qx', 'qy', 'qz'])
    estimate_converted = file_interface.read_euroc_csv_trajectory('temp.txt')
    os.remove('temp.txt')
    return estimate_converted

def dso_aqualoc_workaround(reference, estimate):
    startime = reference.timestamps[0]
    endtime  = reference.timestamps[-1]
    estimate_aux = pd.read_csv(estimate, header = None, delim_whitespace = True, names = ['t', 'x', 'y', 'z', 'qx', 'qy', 'qz', 'qw'] )
    len_estimate = len(estimate_aux['t'].tolist())
    estimate_new_ts = np.linspace(startime*1e9,endtime*1e9, num = len_estimate)
    estimate_aux['#timestamp'] = estimate_new_ts.flatten().tolist()
    
    len_ref = reference.num_poses
    print(len_estimate/len_ref)
    if len_estimate/len_ref > 1:
        estimate_aux = estimate_aux[estimate_aux.index % int(len_estimate/len_ref) == 0]  # Selects every 3rd raw starting from 0
#         estimate_aux = estimate_aux.groupby(estimate_aux.index // int(len_estimate/len_ref)).min()
#         estimate_aux.drop(index=estimate_aux.index[0], axis=0, inplace=True)
    
    estimate_aux.to_csv('temp.txt', header = True, index = False, columns = ['#timestamp', 'x', 'y', 'z', 'qw', 'qx', 'qy', 'qz'])
    estimate_converted = file_interface.read_euroc_csv_trajectory('temp.txt')
    os.remove('temp.txt')
    return estimate_converted   

In [7]:
results = []
result_df = pd.DataFrame() 

---
# DSO

In [8]:
results_dict = {"algorithm": ["DSO", "ORB_SLAM3"],
                "dataset": ["MIMIR"],
                "MIMIR":["SeaFloor/track0","SeaFloor/track1"]}

for algorithm in results_dict["algorithm"]:
    for dataset in results_dict["dataset"]:
        for track in results_dict[dataset]:
            try:
                # find files
                result_dir = os.path.join('..','results',algorithm,dataset,track)
                result_list = glob.glob(result_dir+"/*.txt")
                for result_file in result_list:
                    gt_file = os.path.join('..','groundtruths',dataset,track,'data.tum')            
                    #read GT with EVO
                    gt_evo = file_interface.read_tum_trajectory_file(gt_file)
                    #reat estimate with EVO
                    if algorithm == "DSO":
                        estimated_traj = dso_estimate_workaround(gt_evo, result_file)
                    else:
                        estimated_traj = file_interface.read_tum_trajectory_file(result_file)
                    # save to dataframe
                    df = metrics_dataframe(gt_evo,estimated_traj,algorithm,dataset,track)
                    result_df = pd.concat([result_df, df], axis="columns", join="outer")
            


            except Exception as e: 
                print(e)
                print("failed to execute evaluation in ", dataset, track, " with algorithm ", algorithm)
                print("Check if results file doesnt exist, or it exists and its empty")
                print("---------------------------------------------------------------------------")
result_df

found no matching timestamps between first trajectory and second trajectory with max. time diff 0.01 (s) and time offset 0.0 (s)
failed to execute evaluation in  MIMIR SeaFloor/track0  with algorithm  ORB_SLAM3
Check if results file doesnt exist, or it exists and its empty
found no matching timestamps between first trajectory and second trajectory with max. time diff 0.01 (s) and time offset 0.0 (s)
failed to execute evaluation in  MIMIR SeaFloor/track1  with algorithm  ORB_SLAM3
Check if results file doesnt exist, or it exists and its empty


ORB_SLAM3                                  
                 MIMIR                                  
       SeaFloor/track0                                  
                 ape_t      ape_rad      rpe_t   rpe_rad
rmse         14.148533     2.347167   0.333450  0.068448
mean         11.529719     2.335351   0.232543  0.037992
median        8.864145     2.358483   0.202251  0.025551
std           8.200401     0.235214   0.238982  0.056936
min           0.882573     1.968375   0.022170  0.001096
max          32.566605     3.128178   4.816835  0.885184
sse      109699.182561  3019.036797  60.820373  2.562748

NameError: name 'result_df' is not defined

### EuRoC dataset


In [7]:
algorithm = "DSO"
dataset = "EUROC"
trajectories = ["MH_01_easy","MH_02_easy", "MH_03_medium", "MH_04_difficult", "MH_05_difficult", "V1_01_easy", "V1_02_medium", "V1_03_difficult", "V2_01_easy", "V2_02_medium", "V2_03_difficult"]
for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/euroc"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"mav0","state_groundtruth_estimate0","data.csv")
    ref = file_interface.read_euroc_csv_trajectory(ref_file)
    try:
        est_file = os.getcwd()+"/results/dso/result/euroc/"+trajectory+"/result.txt"

        estimate_fixed = dso_estimate_workaround(ref, est_file)
        df = metrics_dataframe(ref,estimate_fixed,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except Exception as e: 
        print("failed at trajectory ",trajectory)
        print(e)
        continue

### KITTI

In [8]:
algorithm = "DSO"
dataset = "KITTI"
trajectories = ["00", "01", "02", "03", "04", "05", "06", "07","08", "09", "10"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/kitti/data_odometry_poses/dataset/poses"# Edit this to path of dataset w.r.t. current file

    ref_file = os.path.join(abs_dataset_path,trajectory+".txt")
    timestamp_file =os.getcwd()+"/groundtruths/kitti/data_odometry_calib/dataset/sequences/"+trajectory+"/times.txt"
    ref = kitti_poses_and_timestamps_to_trajectory(ref_file, timestamp_file)
    try:
        est_file = os.getcwd()+"/results/dso/result/KITTI/"+trajectory+"/result.txt"
        estimate_fixed = dso_estimate_workaround(ref, est_file)
        df = metrics_dataframe(ref,estimate_fixed,algorithm,dataset,trajectory)    
        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue

failed at trajectory  10


### TUM RGBD

In [9]:
algorithm = "DSO"
dataset = "TUM_RGBD"
trajectories = ["rgbd_dataset_freiburg1_360", "rgbd_dataset_freiburg1_rpy","rgbd_dataset_freiburg3_nostructure_notexture_far","rgbd_dataset_freiburg3_nostructure_texture_near_withloop"]
for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/tum_rgbd"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"groundtruth.txt")
    ref = file_interface.read_tum_trajectory_file(ref_file)
    est_file = os.getcwd()+"/results/dso/result/tum/"+trajectory+"/result.txt"
    try:
        estimate_fixed = dso_estimate_workaround(ref, est_file)
        df = metrics_dataframe(ref,estimate_fixed,algorithm,dataset,trajectory)    
        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue


failed at trajectory  rgbd_dataset_freiburg1_rpy


### AQUALOC

In [10]:
#
        

failed aligning trajectory, trying without align archaeo_sequence_2_raw_data
succeeded
failed aligning trajectory, trying without align archaeo_sequence_3_raw_data
succeeded
failed aligning trajectory, trying without align archaeo_sequence_4_raw_data
succeeded
failed aligning trajectory, trying without align archaeo_sequence_5_raw_data
succeeded
failed aligning trajectory, trying without align archaeo_sequence_6_raw_data
succeeded


---
# ORB_SLAM3 Mono


### KITTI

In [11]:
algorithm = "ORB_SLAM3"
dataset = "KITTI"
trajectories = ["00", "01", "02", "03", "04", "05", "06", "07","08", "09", "10"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/kitti/data_odometry_poses/dataset/poses"# Edit this to path of dataset w.r.t. current file

    ref_file = os.path.join(abs_dataset_path,trajectory+".txt")
    timestamp_file =os.getcwd()+"/groundtruths/kitti/data_odometry_calib/dataset/sequences/"+trajectory+"/times.txt"
    ref = kitti_poses_and_timestamps_to_trajectory(ref_file, timestamp_file)
    
    try:
        est_file = os.getcwd()+"/results/orb-slam/kf_KITTI_"+trajectory+".txt"
        est = file_interface.read_tum_trajectory_file(est_file)
        df = metrics_dataframe(ref,est,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue

## EuRoC dataset


In [12]:
algorithm = "ORB_SLAM3"
dataset = "EUROC"
trajectories = ["MH_01_easy","MH_02_easy", "MH_03_medium", "MH_04_difficult", "MH_05_difficult", "V1_01_easy", "V1_02_medium", "V1_03_difficult", "V2_01_easy", "V2_02_medium", "V2_03_difficult"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/euroc"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"mav0","state_groundtruth_estimate0","data.csv")
    ref = file_interface.read_euroc_csv_trajectory(ref_file)
    try:
        est_file = os.getcwd()+"/results/orb-slam/f_dataset-"+trajectory+"_mono.txt"
        est = file_interface.read_tum_trajectory_file(est_file)
        est.timestamps /= 1e9
        df = metrics_dataframe(ref,est,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue      

## TUM dataset
**Load TUM files with** 3D position and orientation quaternion per line ($x$ $y$ $z$ $q_x$ $q_y$ $q_z$ $q_w$):

In [13]:
dataset = "TUM_RGBD"
algorithm = "ORB_SLAM3"
trajectories = ["rgbd_dataset_freiburg1_360", "rgbd_dataset_freiburg1_rpy","rgbd_dataset_freiburg3_nostructure_notexture_far","rgbd_dataset_freiburg3_nostructure_texture_near_withloop"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/tum_rgbd"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"groundtruth.txt")
    ref = file_interface.read_tum_trajectory_file(ref_file)
    est_file = os.getcwd()+"/results/orb-slam/kf_"+trajectory+".txt"
    try:
        est = file_interface.read_tum_trajectory_file(est_file)
        df = metrics_dataframe(ref,est,algorithm,dataset,trajectory)    
        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue

failed at trajectory  rgbd_dataset_freiburg3_nostructure_notexture_far


### AQUALOC

In [14]:
algorithm = "ORB_SLAM3"
dataset = "AQUALOC"
trajectories = ["1", "2", "3", "4", "5", "6"]
for trajectory in trajectories:
    trajectory_folder = "archaeo_sequence_"+trajectory+"_raw_data"
    abs_dataset_path = os.getcwd()+"/groundtruths/aqualoc"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,"Archaeological_site_sequences","colmap_traj_sequence_"+trajectory+".txt")
    ref = file_interface.read_tum_trajectory_file(ref_file)
    
    tsdataframe = pd.read_csv(os.path.join("/home/olaya/Datasets/Aqualoc/Archaeological_site_sequences",trajectory_folder,"raw_data","img_sequence_"+trajectory+".csv"))
#     print(tsdataframe)
    for i,t in enumerate(ref.timestamps):
        try:
            ref.timestamps[i] = tsdataframe.iloc[int(t)]["#timestamp [ns]"]
        except:
            print(t)
        

    try: 
        est_file =  os.getcwd()+"/results/orb-slam/kf_"+trajectory_folder+".txt"
        est = file_interface.read_tum_trajectory_file(est_file)
        ref.timestamps *= est.timestamps[-1]/ref.timestamps[-1]
    #     print(est.timestamps[0],ref.timestamps[0])
        df = metrics_dataframe(ref,est,algorithm,dataset,trajectory)    
        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print('failed in trajectory', trajectory_folder)
        continue

failed in trajectory archaeo_sequence_4_raw_data


---
# TrianFlow


In [15]:
def trianflow_workaround(reference, estimate):
    startime = reference.timestamps[0]
    endtime = reference.timestamps[-1]
    est = file_interface.read_kitti_poses_file(estimate)
    np.savetxt('temp.txt',np.linspace(startime,endtime,est.num_poses))

    estimate_fixed = kitti_poses_and_timestamps_to_trajectory(estimate, 'temp.txt')
    os.remove('temp.txt')
    return estimate_fixed

### EuRoC

In [16]:
algorithm = "TRIANFLOW"
dataset = "EUROC"
trajectories = ["MH_01_easy","MH_02_easy", "MH_03_medium", "MH_04_difficult", "MH_05_difficult", "V1_01_easy", "V1_02_medium", "V1_03_difficult", "V2_01_easy", "V2_02_medium", "V2_03_difficult"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/euroc"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"mav0","state_groundtruth_estimate0","data.csv")
    ref = file_interface.read_euroc_csv_trajectory(ref_file)
    try:
        est_file = os.getcwd()+"/results/trianflow/results/euroc/"+trajectory+"/trianflow_results.txt"
        estimate_fixed = trianflow_workaround(ref,est_file)
        df = metrics_dataframe(ref,estimate_fixed,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue     

## TUM dataset
**Load TUM files with** 3D position and orientation quaternion per line ($x$ $y$ $z$ $q_x$ $q_y$ $q_z$ $q_w$):

In [17]:
algorithm = "TRIANFLOW"
dataset = "TUM_RGBD"
trajectories = ["rgbd_dataset_freiburg1_rpy","rgbd_dataset_freiburg1_360", "rgbd_dataset_freiburg3_nostructure_notexture_far","rgbd_dataset_freiburg3_nostructure_texture_near_withloop"]


for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/tum_rgbd"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"groundtruth.txt")
    ref = file_interface.read_tum_trajectory_file(ref_file)
    try:    
        est_file = os.getcwd()+"/results/trianflow/results/tum/"+trajectory+"/trianflow_results.txt"
        estimate_fixed = trianflow_workaround(ref,est_file)
        df = metrics_dataframe(ref,estimate_fixed,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue

## KITTI dataset
**Load KITTI files with** entries of the first three rows of  SE(3)  matrices per line (no timestamps)
DFVO records the dataset in KITTI format but with indexes, uncompatible with EVO. Therefore we create a temp file with valid format to load trajectory.

In [18]:
dataset = "KITTI"
algorithm = "TRIANFLOW"
trajectories = ["00", "01", "02", "03", "04", "05", "06", "07","08", "09", "10"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/kitti/data_odometry_poses/dataset/poses"# Edit this to path of dataset w.r.t. current file

    ref_file = os.path.join(abs_dataset_path,trajectory+".txt")
    timestamp_file =os.getcwd()+"/groundtruths/kitti/data_odometry_calib/dataset/sequences/"+trajectory+"/times.txt"
    ref = kitti_poses_and_timestamps_to_trajectory(ref_file, timestamp_file)
    try:    
        est_file = os.getcwd()+"/results/trianflow/results/kitti/"+trajectory+"/trianflow_results.txt"
        estimate_fixed = trianflow_workaround(ref,est_file)
        df = metrics_dataframe(ref,estimate_fixed,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue

### AQUALOC

In [19]:
algorithm = "TRIANFLOW"
dataset = "AQUALOC"
trajectories = ["1", "2", "3", "4", "5", "6"]
for trajectory in trajectories:
    trajectory_folder = "archaeo_sequence_"+trajectory+"_raw_data"
    abs_dataset_path = os.getcwd()+"/groundtruths/aqualoc"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,"Archaeological_site_sequences","colmap_traj_sequence_"+trajectory+".txt")
    ref = file_interface.read_tum_trajectory_file(ref_file)
    try: 
        est_file = os.getcwd()+"/results/trianflow/results/aqualoc/Archaeological_site_sequences/"+trajectory_folder+"/trianflow_results.txt"
        estimate_fixed = dso_estimate_workaround(ref, est_file)
        df = metrics_dataframe(ref,estimate_fixed,algorithm,dataset,trajectory)    
        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print('failed in trajectory', trajectory_folder)
        continue

### MIMIR

In [20]:
algorithm = "TRIANFLOW"
dataset = "MIMIR"
trajectories = ["OceanFloor/track0_dark","OceanFloor/track0_light","SeaFloor/track0","SeaFloor/track1", "SeaFloor_Algae/track0","SeaFloor_Algae/track1"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/mimir"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"auv0","pose_groundtruth","data.tum")
    ref = file_interface.read_tum_trajectory_file(ref_file)
    try:    
        est_file = os.getcwd()+"/results/trianflow/results/mimir/"+trajectory+"/trianflow_results.txt"
        estimate_fixed = trianflow_workaround(ref,est_file)
        df = metrics_dataframe(ref,estimate_fixed,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue

---
# DF-VO

## EuRoC dataset


In [27]:
algorithm = "DFVO"
dataset = "EUROC"
trajectories = ["MH_01_easy","MH_02_easy", "MH_03_medium", "MH_04_difficult", "MH_05_difficult", "V1_01_easy", "V1_02_medium", "V1_03_difficult", "V2_01_easy", "V2_02_medium", "V2_03_difficult"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/euroc"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"mav0","state_groundtruth_estimate0","data.csv")
    ref = file_interface.read_euroc_csv_trajectory(ref_file)
    try:
        est_file = os.getcwd()+"/results/df-vo/result/euroc/"+trajectory+"/"+trajectory+".txt"
        est = file_interface.read_tum_trajectory_file(est_file)
        est.timestamps /= 1e9
        df = metrics_dataframe(ref,est,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue 

FileInterfaceException: csv file /home/freyja/Olaya-data/dev/monocular_visual_slam_survey/evaluation/groundtruths/euroc/MH_01_easy/mav0/state_groundtruth_estimate0/data.csv does not exist

## TUM dataset
**Load TUM files with** 3D position and orientation quaternion per line ($x$ $y$ $z$ $q_x$ $q_y$ $q_z$ $q_w$):

In [26]:
algorithm = "DFVO"
dataset = "TUM_RGBD"
trajectories = ["rgbd_dataset_freiburg3_nostructure_notexture_far","rgbd_dataset_freiburg1_360","rgbd_dataset_freiburg1_rpy", "rgbd_dataset_freiburg3_nostructure_texture_near_withloop"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/tum_rgbd"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"groundtruth.txt")
    ref = file_interface.read_tum_trajectory_file(ref_file)
    try:    
        est_file = os.getcwd()+"/results/df-vo/result/tum/"+trajectory+"/"+trajectory+".txt"
        est = file_interface.read_tum_trajectory_file(est_file)
        df = metrics_dataframe(ref,est,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue

FileInterfaceException: csv file /home/freyja/Olaya-data/dev/monocular_visual_slam_survey/evaluation/groundtruths/tum_rgbd/rgbd_dataset_freiburg3_nostructure_notexture_far/groundtruth.txt does not exist

## KITTI dataset
**Load KITTI files with** entries of the first three rows of  SE(3)  matrices per line (no timestamps)
DFVO records the dataset in KITTI format but with indexes, uncompatible with EVO. Therefore we create a temp file with valid format to load trajectory.

In [23]:
dataset = "KITTI"
algorithm = "DFVO"
trajectories = ["00", "01", "02", "03", "04", "05", "06", "07","08", "09", "10"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/kitti/data_odometry_poses/dataset/poses"# Edit this to path of dataset w.r.t. current file

    ref_file = os.path.join(abs_dataset_path,trajectory+".txt")
    timestamp_file =os.getcwd()+"/groundtruths/kitti/data_odometry_calib/dataset/sequences/"+trajectory+"/times.txt"
    ref = kitti_poses_and_timestamps_to_trajectory(ref_file, timestamp_file)
    try:    
        est_file = os.getcwd()+"/results/df-vo/result/kitti/"+trajectory+"/"+trajectory+".txt"

        # create temporary file
        temp_est_file = pd.read_csv(est_file, header = None, sep=" ", index_col=0)
        temp_est_file.to_csv("temp.txt",sep=" ",index=False, header= False)


        est_fixed = kitti_poses_and_timestamps_to_trajectory("temp.txt", timestamp_file) 

        df = metrics_dataframe(ref,est_fixed,algorithm,dataset,trajectory)    
        #delete temporary file
        os.remove("temp.txt")

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue

### AQUALOC

In [24]:
algorithm = "DFVO"
dataset = "AQUALOC"
trajectories = ["1", "2", "3", "4", "5", "6"]
for trajectory in trajectories:
    trajectory_folder = "archaeo_sequence_"+trajectory+"_raw_data"
    abs_dataset_path = os.getcwd()+"/groundtruths/aqualoc"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,"Archaeological_site_sequences","colmap_traj_sequence_"+trajectory+".txt")
    ref = file_interface.read_tum_trajectory_file(ref_file)      
    try:
        est_file = os.getcwd()+"/results/df-vo/result/aqualoc/Archaeological_site_sequences/"+trajectory_folder+"/"+trajectory_folder+".txt"
        est = file_interface.read_tum_trajectory_file(est_file)
        df = metrics_dataframe(ref,est,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue 

### MIMIR

In [25]:
algorithm = "DFVO"
dataset = "MIMIR"
trajectories = ["OceanFloor/track0_dark","OceanFloor/track0_light","SeaFloor/track0","SeaFloor/track1", "SeaFloor_Algae/track0","SeaFloor_Algae/track1"]

for trajectory in trajectories:
    abs_dataset_path = os.getcwd()+"/groundtruths/mimir"# Edit this to path of dataset w.r.t. current file
    ref_file = os.path.join(abs_dataset_path,trajectory,"auv0","pose_groundtruth","data.tum")
    ref = file_interface.read_tum_trajectory_file(ref_file)
    try:
        est_file = os.getcwd()+"/results/df-vo/result/mimir/"+trajectory+"/result.txt"
        est = file_interface.read_tum_trajectory_file(est_file)
        ref.timestamps*=1e9
        df = metrics_dataframe(ref,est,algorithm,dataset,trajectory)    

        result_df = pd.concat([result_df, df], axis="columns", join="outer")
    except:
        print("failed at trajectory ",trajectory)
        continue 

In [26]:
trajectories = ["","","", "rgbd_dataset_freiburg3_nostructure_texture_near_withloop"]


SyntaxError: invalid syntax (<ipython-input-26-6b8b1023934c>, line 1)

In [196]:
result_df.DFVO.MIMIR["OceanFloor/track0_dark"].rpe_rad.loc["rmse"]

0.16958469459171965

In [ ]:
trajectories = ["OceanFloor/track0_dark","OceanFloor/track0_light","SeaFloor/track0","SeaFloor/track1", "SeaFloor_Algae/track0","SeaFloor_Algae/track1"]


In [ ]:
import pandas as pd
from evo.tools import pandas_bridge

df = pd.DataFrame()
for result in results:
    df = pd.concat((df, pandas_bridge.result_to_df(result)), axis="columns")
df = df.T
df.loc[(df['info'].dataset == "TUM_RGBD")  ]

In [ ]:
# df.loc[(df.info.dataset == "KITTI") & (df.info.algorithm == "DF-VO")]
df.loc[(df['info'].dataset == "TUM_RGBD") & (df['info'].algorithm == "ORB_SLAM3_mono") & (df['info'].trajectory == "rgbd_dataset_freiburg3_nostructure_texture_near_withloop")]["stats"]["rmse"]

In [ ]:
df['info'].columns

In [ ]:
df.to_csv("results.csv")